### Set the Inference server url (replace with your own address)

In [1]:
inference_server_url = "https://mistral-7b-instruct-ollama-test.apps.cluster-d975k.d975k.sandbox3026.opentlc.com"

### Install dependencies

In [2]:
!pip install -q langchain==0.1.9 openai==1.13.3


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip -q install pyOpenSSL


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


### Extract self signed SSL certificate

In [4]:
import os
import socket
import OpenSSL
import socket
from cryptography.hazmat.primitives import serialization
from urllib.parse import urlparse

def save_srv_cert(host, port=443) :
    dst = (host, port)
    sock = socket.create_connection(dst)
    context = OpenSSL.SSL.Context(OpenSSL.SSL.SSLv23_METHOD)
    connection = OpenSSL.SSL.Connection(context, sock)
    connection.set_tlsext_host_name(host.encode('utf-8'))
    connection.set_connect_state()
    try:
        connection.do_handshake()
        certificate = connection.get_peer_certificate()
    except:
        certificate = connection.get_peer_certificate()
    pem_file = certificate.to_cryptography().public_bytes(serialization.Encoding.PEM)
    cert_filename = f"cert-{host}.cer"
    with open(cert_filename, "w") as fout:
        fout.write(pem_file.decode('utf8'))
    return cert_filename

# Extract the hostname"
hostname = urlparse(inference_server_url).netloc
os.environ["SSL_CERT_FILE"] = save_srv_cert(hostname, port=443)

In [5]:
os.environ['REQUESTS_CA_BUNDLE'] = os.environ["SSL_CERT_FILE"]

In [6]:
import ssl
ssl.SSLContext.verify_mode = ssl.VerifyMode.CERT_OPTIONAL
os.environ['PYTHONHTTPSVERIFY'] = '0'

### Preparation for LLM invocation

In [7]:
# Imports
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_community.llms import Ollama
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

In [8]:
# Ensure there is a model loaded for the LLM instance creation
!curl "https://mistral-7b-instruct-ollama-test.apps.cluster-d975k.d975k.sandbox3026.opentlc.com/api/pull" -k -H "Content-Type: application/json" -d '{"name": "mistral"}'

{"status":"pulling manifest"}
{"status":"pulling e8a35b5937a5","digest":"sha256:e8a35b5937a5e6d5c35d1f2a15f161e07eefe5e5bb0a3cdd42998ee79b057730","total":4109853248,"completed":4109853248}
{"status":"pulling 43070e2d4e53","digest":"sha256:43070e2d4e532684de521b885f385d0841030efa2b1a20bafb76133a5e1379c1","total":11356,"completed":11356}
{"status":"pulling e6836092461f","digest":"sha256:e6836092461ffbb2b06d001fce20697f62bfd759c284ee82b581ef53c55de36e","total":42,"completed":42}
{"status":"pulling ed11eda7790d","digest":"sha256:ed11eda7790d05b49395598a42b155812b17e263214292f7b87d15e14003d337","total":30,"completed":30}
{"status":"pulling f9b1e3196ecf","digest":"sha256:f9b1e3196ecfce03ac97468fa0b6d85941fea050c6666c57f5856a8baec6507d","total":483,"completed":483}
{"status":"verifying sha256 digest"}
{"status":"writing manifest"}
{"status":"removing any unused layers"}
{"status":"success"}


### Create the LLM instance

In [9]:
llm_ollama = Ollama(base_url=inference_server_url,
                    model="mistral",
                    top_p=0.92,
                    temperature=0.01,
                    num_predict=512,
                    repeat_penalty=1.03,
                    callbacks=[StreamingStdOutCallbackHandler()]
                   );

### Create the prompt

In [10]:
template="""<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always be as helpful as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

Current conversation:
{history}
Human: {input}
AI:
[/INST]
"""
PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)

### Add some memory for the conversation

In [11]:
memory = ConversationBufferMemory()

### Example using conversation verbose mode. This way we can see the memory buffer

In [12]:
# Verbose mode is intentionally set to True so you can see the prompt and the history from the buffer memory
conversation = ConversationChain(llm=llm_ollama,
                                 prompt=PROMPT,
                                 verbose=True,
                                 memory=memory
                                )

In [13]:
first_input = "Describe Paris in 100 words or less."
conversation.predict(input=first_input);



> Entering new ConversationChain chain...
Prompt after formatting:
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always be as helpful as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

Current conversation:

Human: Describe Paris in 100 words or less.
AI:
[/INST]

 Paris, the City of Light, is an iconic global center for art, fashion, gastronomy, and culture. Known for its historic landmarks including the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral, Paris exudes an enchanting allure. Its picturesque streets are filled with charming cafes, elegant boutiques, and world-re

In [14]:
second_input = "Is there a river?"
conversation.predict(input=second_input);



> Entering new ConversationChain chain...
Prompt after formatting:
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always be as helpful as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

Current conversation:
Human: Describe Paris in 100 words or less.
AI:  Paris, the City of Light, is an iconic global center for art, fashion, gastronomy, and culture. Known for its historic landmarks including the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral, Paris exudes an enchanting allure. Its picturesque streets are filled with charming cafes, elegant boutiques, and world-renowned hau

### Example using non-verbose mode. The output has only the response without the previous chat items.

In [15]:
conversation2 = ConversationChain(llm=llm_ollama,
                                 prompt=PROMPT,
                                 verbose=False,
                                 memory=memory
                                )

In [16]:
# Let's clear the previous conversation first
memory.clear()

In [17]:
first_input = "Can you describe London in 100 words?"
conversation2.predict(input=first_input);

 London, the vibrant capital city of the United Kingdom, is a global cultural melting pot with a rich history. Known for its iconic landmarks like the Tower of London, Buckingham Palace, and the London Eye, it's a city of contrasts, blending modern architecture with ancient history. Its diverse neighborhoods, each with its unique character, offer an array of experiences. From the bustling markets of Camden to the tranquil gardens of Hampstead Heath, London is a city that never sleeps, offering endless opportunities for exploration and discovery.

In [18]:
# This is a follow-up question without context to showcase the conversation memory
second_input = "Is there a river?"
conversation2.predict(input=second_input);

 Absolutely, London is situated on the River Thames, which flows through the heart of the city and is an essential part of its character and history. The river is home to many famous landmarks, including the Houses of Parliament, Tower Bridge, and the iconic London Eye. It's also a popular destination for tourists and locals alike, with numerous boat tours and waterfront activities.